Add train.txt, model.py 

In [1]:
!pip install pillow
!pip install torch
!pip install torchvision

In [41]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [60]:
!unzip /content/drive/MyDrive/dl_495/cross_age_dataset_cleaned_and_resized.zip 

Streaming output truncated to the last 5000 lines.
  inflating: cross_age_dataset_cleaned_and_resized/27_Zachary_Levi_0012.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/59_Rekha_0008.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/55_Lori_Singer_0012.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/36_Ali_Larter_0009.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/32_Steve_Howey_0011.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/32_Rupert_Friend_0012.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/33_Adam_Scott_0009.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/39_Elizabeth_Banks_0009.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/32_Isla_Fisher_0015.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/44_Jack_Black_0008.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/32_Justin_Timberlake_0009.jpg  
  inflating: cross_age_dataset_cleaned_and_resized/22_Kat_Dennings_0010.jpg  
  inflating: cross_a

In [61]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
import os
import torchvision
from model import Simple, Medium, AgeAlexNet

In [62]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'using {device}')

using cuda


In [63]:
class celeb_dataset(Dataset):
    def __init__(self, dataset, path_to_images, train=True):
        self.dataset = dataset
        self.path_to_images = path_to_images
        self.train = train
        
        transform = {"train": transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()]),
            "valid": transforms.Compose([transforms.ToTensor()])}
        
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        X = Image.open(self.path_to_images + self.dataset['name'][index])

        if self.train == True:
            X = self.transform["train"](X)
        else:
            X = self.transform["valid"](X)

        y = torch.tensor(int(self.dataset['class'].iloc[index])).long()

        return X, y

In [68]:
# image_path = '/content/drive/MyDrive/trial/'
image_path = 'cross_age_dataset_cleaned_and_resized/'

df = pd.read_csv('train.txt', sep=' ', header=None)
df.columns = ['name', 'class']

# Changing class labels 
classes_to_covert = list(df['class'])
new = []

for i in classes_to_covert:
    if i == 0 or i == 1:
        new.append(0)
    elif i == 2 or i == 3:
        new.append(1)
    else:
        new.append(2)

df['3_class'] = np.array(new)
df = df.drop(['class'], axis=1)
df.columns = ['name', 'class']

# Train/valid split 
train = df.sample(frac=0.8,random_state=200)
valid = df.drop(train.index)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

# Load data
train_dataset = celeb_dataset(train, image_path, train=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=100)

val_dataset = celeb_dataset(valid, image_path, train=False)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=100)

In [69]:
# Models
model = torchvision.models.vgg16_bn(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
        nn.Linear(25088 , 512),
        nn.BatchNorm1d(512),
        nn.Dropout(0.2),
        nn.Linear(512 , 256),
        nn.Linear(256 , 3))

net = model
net.to(device)

# Loss, optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

overall_loss = []

for epoch in range(1):
    running_loss = 0.0
    for batch, (X,y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        predictions = net(X)
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss}')
            overall_loss.append(loss.item())
            # torch.save(net.state_dict(), class_model_path_Alex + str(i) + '.pt')
            print('fin')

    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss}')
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in val_dataloader:
            X, y = X.to(device), y.to(device)
            outputs = net(X)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    print('validation_accuracy------------->' + str(correct / total))

    running_loss = 0.0

print('finished_training' + str(loss.item()))
print('mean_loss' + str(np.mean(overall_loss)))

FileNotFoundError: ignored

# New Section